In [4]:
import pandas as pd
from shapely.geometry import Polygon, Point
from geopy.distance import distance
from geopy.geocoders import ArcGIS

def generate_square_polygon(center_lat, center_lon, area_km2):
    side_km = area_km2 ** 0.5
    half_side_km = side_km / 2

    # Tạo 4 góc vuông quanh tâm (theo các hướng chính)
    north = distance(kilometers=half_side_km).destination((center_lat, center_lon), 0).latitude
    south = distance(kilometers=half_side_km).destination((center_lat, center_lon), 180).latitude
    east  = distance(kilometers=half_side_km).destination((center_lat, center_lon), 90).longitude
    west  = distance(kilometers=half_side_km).destination((center_lat, center_lon), 270).longitude

    # Theo thứ tự (lon, lat) nếu dùng GeoJSON hoặc shapely
    polygon_coords = [
        (west, south),
        (west, north),
        (east, north),
        (east, south),
        (west, south),  # Đóng polygon
    ]

    return Polygon(polygon_coords)

geolocator = ArcGIS()

In [2]:
# 1. Tạo polygon từ tâm và diện tích
kinhdo = 106.650
vido = 10.8224
dientichkm2 = 10.12
address = 'Nguyễn Sỹ Sách, Quận Tân Bình, Hồ Chí Minh'
polygon = generate_square_polygon(vido, kinhdo, dientichkm2)
location = geolocator.geocode(address)
point = Point(location.longitude, location.latitude)
# 3. Kiểm tra
print("✅ Nằm trong polygon") if polygon.contains(point) else print("❌ Không nằm trong polygon")

✅ Nằm trong polygon


In [5]:
not_sure_district_street = pd.read_csv('data/address-converter.io.vn_not_sure_district_street')

In [6]:
not_sure_district_street

,old_province,old_district,wikipedia,element,street
0,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Bùi Hữu Nghĩa
1,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Cách Mạng Tháng Tám
2,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Thị Nhường
3,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Thùy Trâm
4,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Văn Dầy
...,...,...,...,...,...
2329,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Túc Duyên
2330,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Tố Hữu
2331,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Việt Bắc
2332,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Xuân Hòa


In [7]:
from tqdm import tqdm

In [8]:
for index, row in tqdm(not_sure_district_street.iterrows(), total=len(not_sure_district_street)):
    address = f"{row['street']}, {row['old_district']}, {row['old_province']}"
    location = geolocator.geocode(address)

    if location:
        not_sure_district_street.at[index, 'lat'] = location.latitude
        not_sure_district_street.at[index, 'lon'] = location.longitude
    else:
        not_sure_district_street.at[index, 'lat'] = None
        not_sure_district_street.at[index, 'lon'] = None

100%|██████████| 2334/2334 [33:10<00:00,  1.17it/s] 


In [10]:
not_sure_district_street.to_csv('data/address-converter.io.vn_not_sure_district_street_lat_lon.csv', index=False)

In [11]:
not_sure_district_street

,old_province,old_district,wikipedia,element,street,lat,lon
0,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Bùi Hữu Nghĩa,10.068964,105.757265
1,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Cách Mạng Tháng Tám,10.066906,105.758596
2,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Thị Nhường,10.062843,105.765838
3,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Thùy Trâm,10.063551,105.723182
4,Thành phố Cần Thơ,Quận Bình Thuỷ,https://vi.wikipedia.org/wiki/B%C3%ACnh_Th%E1%...,<ul><li>Bùi Hữu Nghĩa</li>\n<li>Cách Mạng Thán...,Đặng Văn Dầy,10.063120,105.725904
...,...,...,...,...,...,...,...
2329,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Túc Duyên,21.592597,105.847535
2330,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Tố Hữu,21.570571,105.806240
2331,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Việt Bắc,21.594220,105.848170
2332,Tỉnh Thái Nguyên,Thành phố Thái Nguyên,https://vi.wikipedia.org/wiki/Th%C3%A1i_Nguy%C...,<ul><li>Thanh Niên (thuộc khu Quảng trường Võ ...,Xuân Hòa,21.594220,105.848170
